In [1]:
import pandas as pd
import os
import numpy as np
import requests
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime, timedelta
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series
from util import modify_weather_df, modify_entsoe_df, create_lagging_columns, add_future_price_column




# Connecting to Hopsworks and Entose API

In [2]:
if not os.getenv("CI"):  # CI is a common variable set in GitHub Actions
    load_dotenv()
    
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")
entose_api = os.getenv("ENTSOE_API")

# Check if keys are loaded properly
if not hopsworks_api:
    raise ValueError("HOPSWORKS_API_KEY is not set.")
if not entose_api:
    raise ValueError("ENTSOE_API is not set.")

os.environ["ENTSOE_API"] = entose_api
os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")


2025-01-06 21:55:13,432 INFO: Initializing external client
2025-01-06 21:55:13,432 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 21:55:14,925 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [3]:
start_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

### Get weather forecast

In [4]:
forecast = get_weather_forecast("Stockholm", "2022-11-01", "2025-01-03", 59.3294, 18.0687)

formatted_forecast_df = process_weather_data(forecast)
formatted_forecast_df = modify_weather_df(formatted_forecast_df)
formatted_forecast_df = create_lagging_columns(formatted_forecast_df)
print(formatted_forecast_df.dtypes)

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
number of rows before dropping na:  168
number of rows after dropping na:  167
date                   datetime64[ns]
temperature_2m                float32
precipitation                 float32
wind_speed_100m               float32
temperature_2m_lag            float32
precipitation_lag             float32
wind_speed_100m_lag           float32
dtype: object


In [5]:
# price_fg = fs.get_feature_group(
#     name='electricity_price_data_3',
#     version=1,
# )
weather_fg = fs.get_feature_group(
    name='weather_data_3',
    version=1,
)
entsoe_fg = fs.get_feature_group(
    name='entsoe_data_3',
    version=1,
)

In [7]:

#formatted_forecast_df['date'] = pd.to_datetime(formatted_forecast_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)

In [6]:
print(formatted_forecast_df.isna().sum())

date                   0
temperature_2m         0
precipitation          0
wind_speed_100m        0
temperature_2m_lag     0
precipitation_lag      0
wind_speed_100m_lag    0
dtype: int64


In [7]:
formatted_forecast_df = formatted_forecast_df.dropna()
weather_fg.insert(formatted_forecast_df)

2025-01-06 21:55:31,733 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393654


Uploading Dataframe: 100.00% |██████████| Rows 167/167 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_3_1_offline_fg_materialization/executions


(Job('weather_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_lag",
           "min_value": -5.0,
           "max_value": 500.0
         },
         "meta": {
           "expectationId": 696119
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 167,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T08:55:31.000732Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
   

In [8]:
# price = get_todays_data("SE3")
# price['time_start'] = price['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
# print(price)
# print(price.dtypes)

In [8]:
entsoe_df = fetch_historical_data(entose_api, start_date, end_date)

[DEBUG] Successfully fetched load data for finland.
[DEBUG] Successfully fetched load data for norway.
[DEBUG] Successfully fetched load data for denmark.
[DEBUG] Successfully fetched cross-border flows SE3 to finland.
[DEBUG] Successfully fetched cross-border flows finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to norway.
[DEBUG] Successfully fetched cross-border flows norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to denmark.
[DEBUG] Successfully fetched cross-border flows denmark to SE3.


In [9]:
print(entsoe_df.dtypes)

load_se3                                            float64
load_finland                                        float64
load_norway                                         float64
load_denmark                                        float64
prices                                              float64
total_generation_Biomass                            float64
total_generation_Fossil Gas                         float64
total_generation_Fossil Hard coal                   float64
total_generation_Fossil Oil                         float64
total_generation_Hydro Run-of-river and poundage    float64
total_generation_Other renewable                    float64
total_generation_Solar                              float64
total_generation_Waste                              float64
total_generation_Wind Offshore                      float64
total_generation_Wind Onshore                       float64
flows_se3_to_finland                                float64
flows_finland_to_se3                    

In [10]:

entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_df.columns = entsoe_df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])
entsoe_df['date'] = pd.to_datetime(entsoe_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)
entsoe_df = entsoe_df.dropna()
print(entsoe_df.dtypes)
print(entsoe_df.tail(5))

load_se3                                                   float64
load_finland                                               float64
load_norway                                                float64
load_denmark                                               float64
prices                                                     float64
total_generation_biomass                                   float64
total_generation_fossil_gas                                float64
total_generation_fossil_hard_coal                          float64
total_generation_fossil_oil                                float64
total_generation_hydro_run_of_river_and_poundage           float64
total_generation_other_renewable                           float64
total_generation_solar                                     float64
total_generation_waste                                     float64
total_generation_wind_offshore                             float64
total_generation_wind_onshore                              flo

In [11]:
entsoe_df = modify_entsoe_df(entsoe_df)
entsoe_df = create_lagging_columns(entsoe_df)
entsoe_df = add_future_price_column(entsoe_df)

number of rows before dropping na:  21
number of rows after dropping na:  20


In [12]:
print(entsoe_df.dtypes)
entsoe_df

prices                            float64
date                       datetime64[ns]
imported_energy                   float64
total_load                        float64
total_generation_se               float64
prices_lag                        float64
imported_energy_lag               float64
total_load_lag                    float64
total_generation_se_lag           float64
future_price                      float64
dtype: object


,prices,date,imported_energy,total_load,total_generation_se,prices_lag,imported_energy_lag,total_load_lag,total_generation_se_lag,future_price
index,,,,,,,,,,
2025-01-06 00:00:00+00:00,19.26,2025-01-06 01:00:00,-1983.0,32226.25,4557.0,27.52,-1408.0,32692.25,4692.0,11.35
2025-01-06 01:00:00+00:00,11.35,2025-01-06 02:00:00,-1998.0,32200.25,4383.0,19.26,-1983.0,32226.25,4557.0,9.20
2025-01-06 02:00:00+00:00,9.20,2025-01-06 03:00:00,-1990.0,32018.00,4259.0,11.35,-1998.0,32200.25,4383.0,10.00
2025-01-06 03:00:00+00:00,10.00,2025-01-06 04:00:00,-1932.0,31894.25,4128.0,9.20,-1990.0,32018.00,4259.0,14.81
2025-01-06 04:00:00+00:00,14.81,2025-01-06 05:00:00,-1903.0,31948.25,4155.0,10.00,-1932.0,31894.25,4128.0,23.82
2025-01-06 05:00:00+00:00,23.82,2025-01-06 06:00:00,-2351.0,32692.50,4270.0,14.81,-1903.0,31948.25,4155.0,31.72
2025-01-06 06:00:00+00:00,31.72,2025-01-06 07:00:00,-1466.0,33867.00,4290.0,23.82,-2351.0,32692.50,4270.0,39.58
2025-01-06 07:00:00+00:00,39.58,2025-01-06 08:00:00,-1028.0,34496.75,4467.0,31.72,-1466.0,33867.00,4290.0,36.08
2025-01-06 08:00:00+00:00,36.08,2025-01-06 09:00:00,-1010.0,34833.50,4521.0,39.58,-1028.0,34496.75,4467.0,34.69


In [13]:
entsoe_fg.insert(entsoe_df)

2025-01-06 21:56:18,293 INFO: 	9 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393655


Uploading Dataframe: 100.00% |██████████| Rows 20/20 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: entsoe_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_3_1_offline_fg_materialization/executions


(Job('entsoe_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "prices_lag",
           "min_value": -1000,
           "max_value": 1000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 696127
         }
       },
       "result": {
         "observed_value": 9.2,
         "element_count": 20,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T08:56:18.000293Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_m

In [14]:
print(entsoe_df.dtypes)
entsoe_df

prices                            float64
date                       datetime64[ns]
imported_energy                   float64
total_load                        float64
total_generation_se               float64
prices_lag                        float64
imported_energy_lag               float64
total_load_lag                    float64
total_generation_se_lag           float64
future_price                      float64
dtype: object


,prices,date,imported_energy,total_load,total_generation_se,prices_lag,imported_energy_lag,total_load_lag,total_generation_se_lag,future_price
index,,,,,,,,,,
2025-01-06 00:00:00+00:00,19.26,2025-01-06 01:00:00,-1983.0,32226.25,4557.0,27.52,-1408.0,32692.25,4692.0,11.35
2025-01-06 01:00:00+00:00,11.35,2025-01-06 02:00:00,-1998.0,32200.25,4383.0,19.26,-1983.0,32226.25,4557.0,9.20
2025-01-06 02:00:00+00:00,9.20,2025-01-06 03:00:00,-1990.0,32018.00,4259.0,11.35,-1998.0,32200.25,4383.0,10.00
2025-01-06 03:00:00+00:00,10.00,2025-01-06 04:00:00,-1932.0,31894.25,4128.0,9.20,-1990.0,32018.00,4259.0,14.81
2025-01-06 04:00:00+00:00,14.81,2025-01-06 05:00:00,-1903.0,31948.25,4155.0,10.00,-1932.0,31894.25,4128.0,23.82
2025-01-06 05:00:00+00:00,23.82,2025-01-06 06:00:00,-2351.0,32692.50,4270.0,14.81,-1903.0,31948.25,4155.0,31.72
2025-01-06 06:00:00+00:00,31.72,2025-01-06 07:00:00,-1466.0,33867.00,4290.0,23.82,-2351.0,32692.50,4270.0,39.58
2025-01-06 07:00:00+00:00,39.58,2025-01-06 08:00:00,-1028.0,34496.75,4467.0,31.72,-1466.0,33867.00,4290.0,36.08
2025-01-06 08:00:00+00:00,36.08,2025-01-06 09:00:00,-1010.0,34833.50,4521.0,39.58,-1028.0,34496.75,4467.0,34.69
